# Classification: Softmax-Eis für einen one-hot day

## Klassifizierung mit neuronalen Netzen
In diesem Kapitel werden Sie die Funktionsweise eines Klassifizierers kennenlernen und einen solchen Klassifizierer manuell trainieren, der mit einem einzelnen Wert arbeitet. Sie werden den Klassifikator Schritt für Schritt verbessern und dabei grundlegende Konzepte der Klassifizierung verstehen.
Anschließend werden Sie mit Hilfe der automatischen Backpropagation ein mehrschichtiges neuronales Netz trainieren, das ein logisches Gatter nachahmt.

### Einführung
Beim maschinellen Lernen und in der Statistik ist die Klassifizierung das Problem. Hierzu wird eine Beobachtung einem Satz von Kategorien (Unterpopulationen) zugeordnet, und zwar auf der Grundlage eines Trainingssatzes von Daten, die Beobachtungen (oder Instanzen) enthalten und deren Kategorie-Zugehörigkeit bekannt ist. Beispiele hierfür sind die Zuordnung einer gegebenen E-Mail zur Klasse "Spam" oder "Nicht-Spam" oder die Diagnosevergabe an einen Patienten auf der Grundlage von beobachteten Merkmalen des Patienten (Geschlecht, Blutdruck, Vorhandensein oder Fehlen bestimmter Symptome usw.). [1]

Ein Klassifizierungsprozess erfordert einen Datensatz, der in verschiedene Kategorien unterteilt ist. Ein Klassifikator kann mit diesem Datensatz trainiert werden, indem er die Beziehung zwischen bestimmten Eigenschaften der Eingabedaten und der zugehörigen Kategorien erlernt.
Der Prozess zur Klassifizierung neuer Daten ist dabei ähnlich wie der im Kapitel "Regression" vorgestellt, jedoch können je nach Anwendung zusätzliche Rechenschritte hinzugefügt werden.
Ein bekanntes Klassifizierungsproblem, das mit neuronalen Netzen gelöst werden kann, ist die Bilderkennung (siehe Abbildung 1).



<img src="https://github.com/KI-Campus/AMALEA/blob/master/Woche%204/images/neural_network_classification.png?raw=1" />
<p style="text-align: center;">
    Abb. 1 - Bilderkennung durch ein neuronales Netz
</p>

Führen Sie die folgenden Zellen aus, um die erforderlichen Bibliotheken zu importieren und eine ReLU-, MSE-Verlustfunktion und eine SimpleNeuron-Klasse zu definieren.

Wenn Sie Ihren eigenen Computer verwenden, müssen Sie wahrscheinlich die Bibliothek <code>plotly</code> installieren.
Dies können Sie mit <code>conda install plotly</code> in der Befehlszeile von Anaconda  erledigen.

In [1]:
# do not change
import numpy as np
from ipywidgets import interact, Layout, FloatSlider
import plotly.offline as plotly
import plotly.graph_objs as go
import time
import threading

In [2]:
# do not change
def relu(input_val:float)->float:
    return np.where(input_val > 0, input_val, 0.0)

In [3]:
# do not change
def mean_squared_loss(predictions:list, solutions:list)->float:
    total_squared_loss = np.sum(np.subtract(predictions, solutions)**2) #np allows to handle both values and lists
    mean_squared_loss = total_squared_loss/len(predictions)
    return mean_squared_loss

In [4]:
# do not change
class SimpleNeuron:
    def __init__(self, plot):
        self.plot = plot
        self.plot.register_neuron(self) #hey plot, remember me

    def set_values(self, weight:float, bias:float):
        self.weight = weight
        self.bias = bias
        self.plot.update() #hey plot, I have changed, redraw my output

    def get_weight(self) -> float:
        return self.weight

    def get_bias(self)->float:
        return self.bias

    def compute(self, x:float)->float:
        self.activation = np.dot(self.weight, x) + self.bias
        return self.activation

In [5]:
# do not change
# an Interactive Plot monitors the activation of a neuron or a neural network
class Interactive2DPlot:
    def __init__(
        self, points_red, points_blue, ranges, loss_function=mean_squared_loss, loss_string="Loss", width=800, height=400, margin=dict(t=0, l=170),
        draw_time=0.1
    ):
        self.idle = True
        self.points_red = points_red
        self.points_blue = points_blue
        self.draw_time = draw_time
        self.loss_function = loss_function
        self.loss_string = loss_string

        self.x = np.arange(ranges["x"][0], ranges["x"][1], 0.01)
        self.y = np.arange(ranges["y"][0], ranges["y"][1], 0.01)

        self.layout = go.Layout(
            xaxis=dict(title="Neck height in m", range=ranges["x"]),
            yaxis=dict(title="y", range=ranges["y"]),
            width=width,
            height=height,
            showlegend=False,
            margin=margin,
        )
        self.trace = go.Scatter(x=self.x, y=self.y)

        self.plot_points_red = go.Scatter(
            x=points_red["x"], y=points_red["y"], mode="markers", marker=dict(color='rgb(255, 0, 0)', size=10)
        )
        self.plot_points_blue = go.Scatter(
            x=points_blue["x"],
            y=points_blue["y"],
            mode="markers",
            marker=dict(color='rgb(0, 0, 255)', size=10, symbol="square"),
        )

        self.plot_point_new = go.Scatter(
            x=[], y=[], mode="markers", marker=dict(size=20, symbol="star", color='rgb(0,0,0)')
        )

        self.data = [self.trace, self.plot_points_red, self.plot_points_blue, self.plot_point_new]
        self.plot = go.FigureWidget(self.data, self.layout)

    def register_neuron(self, neuron):
        self.neuron = neuron

    def redraw(self):
        self.idle = False
        time.sleep(self.draw_time)
        self.plot.data[0].y = self.neuron.compute(self.x)
        self.idle = True

    def update(self):
        loss_red = self.loss_function(self.neuron.compute(self.points_red["x"]), self.points_red["y"])
        loss_blue = self.loss_function(self.neuron.compute(self.points_blue["x"]), self.points_blue["y"])
        print(self.loss_string,": {:0.3f}".format((loss_red + loss_blue) / 2))

        if self.idle:
            thread = threading.Thread(target=self.redraw)
            thread.start()

## Von der Regression zur Klassifikation

### Lineare Regression

Sie arbeiten auf einem Bauernhof mit Schafen und Lamas, die in getrennten Gehegen grasen. Letzte Nacht hat der Schäfer jedoch vergessen, das Tor zwischen den beiden Gehegen zu schließen. Die Lamas und Schafe sind nun vermischt und müssen wieder getrennt werden. Sie denken sich sofort eine auf maschinellem Lernen basierende Lösung aus, um die Schafe wieder von den Lamas zu trennen: Sie gehen davon aus, dass Lamas von Schafen unterschieden werden können, indem Sie den Abstand von der Oberseite des Kopfes bis zur Wirbelsäule messen. Da Lamas deutlich längere Hälse aufweisen. Mit einem LIDAR-Scanner wird die Halshöhe autonom gemessen und die Tiere werden mit einem Futterköder und einem elektronischen Drehkreuz, das nur Lamas durchlässt, getrennt.


<img src="https://github.com/KI-Campus/AMALEA/blob/master/Woche%204/images/neck_heights.png?raw=1" />
<p style="text-align: center;">
    Abb. 2 - Konzept der Halshöhenmessung
</p>

Um Datenpunkte zu sammeln, gehen Sie mit einem Maßband auf das Feld und messen die Halshöhen einiger Schafe und Lamas. Sie legen zwei Kategorien fest: "0" für Schafe und "1" für Lamas. (Siehe Tabelle 1)

Die meisten Lamas sind erwachsen und haben lange Hälse, aber es gibt auch einige junge Lamas mit kleineren Hälsen. Da ihre Hälse jedoch immer noch länger sind als die der Schafe, gehen Sie davon aus, dass dies kein Problem sein wird.

|  Tier | Halshöhe  | Kategorie  |
|---------|--------------|-----------|
| Schaf #1| 0.20m        |0          |
| Schaf #2| 0.23m        |0          |
| Schaf #3| 0.28m        |0          |
| Schaf #4| 0.32m        |0          |
| Schaf #5| 0.35m        |0          |
| Lama #1| 0.55m        |1          |
| Lama #2| 0.68m        |1          |
| Lama #3| 0.74m        |1          |
| Lama #4| 0.83m        |1          |
| Lama #5| 0.95m        |1          |

<p style="text-align: center;">
    Tabelle. 1 - Ihre Data-Mining-Ergebnisse
</p>

#### Trainieren eines linearen Regressionsneurons von Hand
Der Einfachheit halber beginnen Sie damit, ein einzelnes Neuron als Klassifikator zu verwenden. Führen Sie die beiden folgenden Zellen aus, um die Data-Mining-Punkte zu definieren und ein Diagramm anzuzeigen.

In [6]:
# do not change
points_sheep = dict(
              x=[ 0.20, 0.23, 0.28, 0.32, 0.35],
              y=[ 0, 0, 0, 0, 0]
             )

points_llamas = dict(
              x=[ 0.55, 0.68, 0.74, 0.83, 0.95],
              y=[ 1,  1, 1, 1, 1]
             )

ranges = dict(x=[-0.1, 1.25], y=[-0.5, 1.4])
slider_layout = Layout(width="90%")

In [18]:
# do not change
plot1 = Interactive2DPlot(points_sheep, points_llamas, ranges, loss_string="Mean Squared Loss")
neuron1 = SimpleNeuron(plot1)

interact(
    neuron1.set_values,
    weight=FloatSlider(min=-2, max=4, step=0.1, layout = slider_layout),
    bias=FloatSlider(min=-1, max=1, step=0.1, layout = slider_layout),
)

plot1.plot

interactive(children=(FloatSlider(value=0.0, description='weight', layout=Layout(width='90%'), max=4.0, min=-2…

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '0876cf4b-42ff-4350-970a-7ae5493530b0',
              'x': array([-1.00000000e-01, -9.00000000e-02, -8.00000000e-02, -7.00000000e-02,
                          -6.00000000e-02, -5.00000000e-02, -4.00000000e-02, -3.00000000e-02,
                          -2.00000000e-02, -1.00000000e-02, -5.55111512e-17,  1.00000000e-02,
                           2.00000000e-02,  3.00000000e-02,  4.00000000e-02,  5.00000000e-02,
                           6.00000000e-02,  7.00000000e-02,  8.00000000e-02,  9.00000000e-02,
                           1.00000000e-01,  1.10000000e-01,  1.20000000e-01,  1.30000000e-01,
                           1.40000000e-01,  1.50000000e-01,  1.60000000e-01,  1.70000000e-01,
                           1.80000000e-01,  1.90000000e-01,  2.00000000e-01,  2.10000000e-01,
                           2.20000000e-01,  2.30000000e-01,  2.40000000e-01,  2.50000000e-01,
                           2.60000000e-01,  2

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

<div class="alert alert-block alert-success">
<b>Frage 4.4.1:</b> Variieren Sie die obigen Schieberegler für Gewicht und Bias. Was ist eine Kombination aus Gewicht und Bias, die zu einem Verlust < 0,05 führt?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


***
#### Auf dem Weg zu einem diskreten Klassifikator
Nun wollen wir unser trainiertes Neuron verwenden, um neue Halshöhen zu klassifizieren. Dazu müssen wir ein Programm schreiben, das eine Halshöhe entgegennimmt und ausgibt, was das trainierte Neuron darüber denkt. Der Klassifikator wird auch die neue Halshöhe aufzeichnen. Führen Sie die folgende Box aus, um die Werte aus der vorherigen Aufgabe zu erhalten.

In [19]:
# do not change
# a duplicate of the last plot, so you don't have to scroll
plot2 = Interactive2DPlot(points_sheep, points_llamas, ranges, loss_string="Mean Squared Loss")
neuron2 = SimpleNeuron(plot2)
neuron2.set_values(neuron1.get_weight(), neuron1.get_bias()) #get your values from last task

plot2.plot

Mean Squared Loss : 0.042


FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'dfddfe0c-04e8-4b88-a41d-2face70ebe9a',
              'x': array([-1.00000000e-01, -9.00000000e-02, -8.00000000e-02, -7.00000000e-02,
                          -6.00000000e-02, -5.00000000e-02, -4.00000000e-02, -3.00000000e-02,
                          -2.00000000e-02, -1.00000000e-02, -5.55111512e-17,  1.00000000e-02,
                           2.00000000e-02,  3.00000000e-02,  4.00000000e-02,  5.00000000e-02,
                           6.00000000e-02,  7.00000000e-02,  8.00000000e-02,  9.00000000e-02,
                           1.00000000e-01,  1.10000000e-01,  1.20000000e-01,  1.30000000e-01,
                           1.40000000e-01,  1.50000000e-01,  1.60000000e-01,  1.70000000e-01,
                           1.80000000e-01,  1.90000000e-01,  2.00000000e-01,  2.10000000e-01,
                           2.20000000e-01,  2.30000000e-01,  2.40000000e-01,  2.50000000e-01,
                           2.60000000e-01,  2

In [15]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

<div class="alert alert-block alert-success">
<b>Aufgabe 4.4.2:</b> Versuchen Sie, einen Klassifikator mit nur einem linearen Neuron zu implementieren. (Ja, eine fast aussichtslose Aufgabe, aber das wird später noch Sinn ergeben). <br> Vervollständigen Sie den folgenden Python-Code sodass Sie das Ergebnis der Klassifizierung als Variable <code>classification_result</code> erhalten.
<ul>
    <li> Das Klassifikationsergebnis soll die Ausgabe von neuron2 auf Basis einer neuen Halshöhe sein</li>
    <li> Sie sollten nicht mehr als 1 Zeile Code hinzufügen müssen </li>
    <li> Schauen Sie sich nach dem Ausführen den Stern im obigen Diagramm an. Er stellt die aktuelle Eingabe/Ausgabe für die neue Halslänge dar</li>
</ul>

</div>

In [24]:
new_neck_height = 0.55  # this value shall be varied to answer the questions below

#classification_result = ??

#STUDENT CODE HERE
classification_result = neuron2.compute(new_neck_height)
#STUDENT CODE until HERE

plot2.plot.data[3].x = [new_neck_height] #update plot
plot2.plot.data[3].y = [classification_result]

print("Result:", classification_result)

Result: 0.5800000000000001


<div class="alert alert-block alert-success">
<b>Frage 4.4.3:</b> Welchen Klassifizierungswert hat das kleinste Lama? (führen Sie die Zelle oben aus und ändern Sie <code>new_neck_height</code>)  
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 4.4.4:</b> Welchen Klassifizierungswert hat ein Tier mit einer Halshöhe von 0,1m?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 4.4.5:</b> Welchen Klassifizierungswert hat ein Tier mit einer Halshöhe von 0,9 m?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 4.4.6:</b> Warum ist der Klassifikationswert kontinuierlich, obwohl die Trainingsdaten nur zwei diskrete Werte hatten?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 4.4.7:</b> Wie würden Sie diesen kontinuierlichen Klassifikationswert interpretieren? Versuchen Sie, ihn in wenigen Worten zu beschreiben, es gibt mehrere richtige Antworten.
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 4.4.8:</b> Ihr Neuron gibt einen kontinuierlichen Wert aus, aber wir brauchen eine diskrete Ausgabe, die eindeutig entweder "Lama" oder "Schaf" vorhersagt. Um das zu erreichen, fügen Sie der Ausgabe des Neurons eine einfache Entscheidung hinzu. Die Entscheidung sollte ungefähr genauso empfindlich gegenüber Lamas wie gegenüber Schafen sein. Welche Neuronenausgabe (y-Wert) würden Sie als Schwellenwert wählen und warum? (mehrere  richtige Antworten)
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 4.4.9:</b> Sie möchten mehr Daten zu Ihrem Modell hinzufügen, um dessen Performanz zu verbessern. Während Sie mehr Daten sammeln, finden Sie ein kleines Lama mit einer Halshöhe von 0,40 m in Ihrem Datensatz. Nachdem Sie Ihr Modell auf den neuen Daten trainiert haben, entscheidet Ihr diskreter Klassifikator, dass dieses kleine Lama ein Schaf ist. (Zur Erinnerung: die Entscheidung am Ende erhält nur den y-Wert). Warum ist es in diesem Fall problematisch, ein <b>lineares</b> Regressionsmodell für die diskrete Klassifikation zu verwenden? Welche Eigenschaft der Näherungsfunktion sollte anders sein?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 4.4.10:</b> Sie entscheiden, dass das manuelle Hinzufügen einer diskreten Entscheidung am Ende Ihres Netzes eine unpraktische Idee ist. Es wäre besser, das lineare Neuron zu verbessern, indem man eine Heaviside-Step-Funktion als Aktivierungsfunktion hinzufügt, so wie man eine ReLu-Funktion hinzufügt. Dann könnte das Training automatisiert werden und die richtige Schwelle automatisch gefunden werden. Was ist das Problem bei diesem Ansatz, wenn wir weiterhin den Backpropagation-Algorithmus verwenden wollen?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b>
</div>

***
### Logistische Regression

Beim maschinellen Lernen ist die logistische Verteilung eine gängige Annahme für eine unbekannte Zwei-Klassen-Wahrscheinlichkeitsverteilung.[2]
Ihre kumulierte Funktion ist die logistische Funktion, von der die Sigmoidfunktion der am häufigsten verwendete Spezialfall ist (siehe Abb. 3).
Die Sigmoidfunktion ermöglicht es einem Modell, die meisten natürlich vorkommenden Wahrscheinlichkeitsverteilungen zu erfassen.[3] (Weiterführende Literatur: siehe Abschnitt "Weiterführende Literatur" am Ende des Dokuments)

In der Einleitung zu Aufgabe 2.1 haben wir den Halslängen entsprechende Bezeichnungen gegeben. "0" für Schaf und "1" für Lama.
Hier können wir die Ausgabe des Neurons als die "Lama-Wahrscheinlichkeit" interpretieren. Zum Beispiel: Ein Ausgang von 1 bedeutet "100%" Lama-Wahrscheinlichkeit und ein Ausgang von 0,2 bedeutet "20%" Lama-Wahrscheinlichkeit und so weiter.

<img src="https://github.com/KI-Campus/AMALEA/blob/master/Woche%204/images/sigmoid.png?raw=1" />
<p style="text-align: center;">
    Abb. 3 - Sigmoid-Funktion
</p>


Führen Sie die folgende Zelle aus, um eine Sigmoidfunktion zu definieren.

In [25]:
# do not change
def sigmoid(x:float)->float:
    return 1 / (1 + np.exp(-x))

<div class="alert alert-block alert-success">
<b>Aufgabe 4.4.11:</b> Vervollständigen Sie den Code und trainieren sie das Neuron. Modifizieren Sie die Klasse <code>SigmoidNeuron</code>, um eine Sigmoid-Funktion auf die endgültige Ausgabe anzuwenden.

</div>

In [28]:
class SigmoidNeuron(SimpleNeuron): #inheriting from SimpleNeuron,
                                   #all functions stay the same unless they are specified here

    def compute(self, x:float)->float:
        # STUDENT CODE HERE
        self.activation = sigmoid(np.dot(self.weight, x) + self.bias)

        # STUDENT CODE until HERE
        return self.activation

In [29]:
# do not change
classification_plot_sig = Interactive2DPlot(points_llamas, points_sheep, ranges, loss_string="Mean Squared Loss")

our_sig_neuron = SigmoidNeuron(classification_plot_sig)

interact(
    our_sig_neuron.set_values,
    weight=FloatSlider(min=-50, max=200, step=0.1, layout = slider_layout),
    bias=FloatSlider(min=-50, max=50, step=0.1, layout = slider_layout),
)

classification_plot_sig.plot

interactive(children=(FloatSlider(value=0.0, description='weight', layout=Layout(width='90%'), max=200.0, min=…

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'bef346e0-af67-4089-92a4-8d50931cafd2',
              'x': array([-1.00000000e-01, -9.00000000e-02, -8.00000000e-02, -7.00000000e-02,
                          -6.00000000e-02, -5.00000000e-02, -4.00000000e-02, -3.00000000e-02,
                          -2.00000000e-02, -1.00000000e-02, -5.55111512e-17,  1.00000000e-02,
                           2.00000000e-02,  3.00000000e-02,  4.00000000e-02,  5.00000000e-02,
                           6.00000000e-02,  7.00000000e-02,  8.00000000e-02,  9.00000000e-02,
                           1.00000000e-01,  1.10000000e-01,  1.20000000e-01,  1.30000000e-01,
                           1.40000000e-01,  1.50000000e-01,  1.60000000e-01,  1.70000000e-01,
                           1.80000000e-01,  1.90000000e-01,  2.00000000e-01,  2.10000000e-01,
                           2.20000000e-01,  2.30000000e-01,  2.40000000e-01,  2.50000000e-01,
                           2.60000000e-01,  2

<div class="alert alert-block alert-success">
<b>Frage 4.4.12:</b> Nennen Sie ein Beispiel für eine optimale Kombination aus Gewicht und Bias
</div>

<div class="alert lock alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 4.4.13:</b> Welchen Vorteil hat ein Klassifikator, der auch eine Wahrscheinlichkeit ausgibt, im Vergleich zu einem Klassifikator, der nur einen binären Ja/Nein-Wert ausgibt? (ein paar Worte)
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 4.4.14:</b> Nennen Sie ein Beispiel, wie wir die zusätzlichen Wahrscheinlichkeitsinformationen nutzen können, um die Genauigkeit unseres Trennungsprozesses zu erhöhen
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


### Kreuzentropie/Logarithmischer Verlust (engl.  Cross Entropy/Logarithmic Loss):
Die gängigste Verlustfunktion für die Klassifizierung ist der Kreuzentropieverlust, auch logarithmischer Verlust genannt (im Kontext des maschinellen Lernens bezeichnen beide Begriffe das gängigste Gleiche). Im Spezialfall von zwei Kategorien wird der Verlust als binäre Kreuzentropie bezeichnet. Der binäre Kreuzentropieverlust zwischen einem tatsächlichen Datenwert $y$ und einem vorhergesagten Wert $p$ wird wie folgt berechnet:

\begin{align}
-[y \cdot log(p)+(1-y)\cdot log(1-p)]
\end{align}

Auf diese Weise wird der Durchschnitt aller Datenpunkte unter Berücksichtigung dieses Verlustes berechnet.
Es stellt sich heraus, dass die Ableitung eines logarithmischen Verlustes unter Verwendung einer "One-Hot Kodierung" (unten erklärt) einfach der Lösungsvektor subtrahiert von der Netzwerkausgabe ist, was die Arbeit damit sehr einfach macht.
**Hinweis:** Der Kreuzentropieverlust kann nur verwendet werden, wenn die Ausgabewerte zwischen 0 und 1 liegen.

<img src="https://github.com/KI-Campus/AMALEA/blob/master/Woche%204/images/cross_entropy.png?raw=1" />
<p style="text-align: center;">
        Abb. 4 - Logarithmische / Kreuzentropie-Verlustfunktion

</p>

<div class="alert alert-block alert-success">
<b>Aufgabe 4.4.15:</b> Berechnen Sie den quadratischen und den Kreuz-Entropieverlust. Kopieren Sie die Tabelle und füllen Sie das ??? als Antwort unten aus (Markdown ist gut geeignet, um die Tabelle darzustellen). Verwenden Sie die Zellen unten für die Berechnungen.

</div>


| Eingabe | Lama-Wahrscheinlichkeit | Quadratischer Verlust | Kreuzentropieverlust |
|---------------|--------------------|----------------------|----------------------|
| Lama(1) | 0.99 |??????                |????                  |
| Schafe(0) | 0.6 |????                  |????                  |
| sheep(0) | 0.95 |??????                |????                  |
| sheep(0) | 0.999999 |????????              |?????                 |

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>









In [30]:
# do not change
def cross_entropy_loss(predictions:list, solutions:list) -> float:
    predictions += 1e-15 #in order to prevent log(0)
    total_loss = np.sum(-(solutions*np.log(predictions)+(1-solutions)*np.log(1-predictions)))
    avg_loss = total_loss/len(predictions)
    return avg_loss

In [31]:
predicted = np.array([0.1]) #insert here
actual = np.array([0.1]) #insert here


print("mean squared loss: {:0.4f}".format(mean_squared_loss(predicted,actual)))
print("cross entropy loss: {:0.4f}".format(cross_entropy_loss(predicted,actual)))

mean squared loss: 0.0000
cross entropy loss: 0.3251


<div class="alert alert-block alert-success">
<b>Frage 4.4.16:</b> Wie unterscheiden sich die Ziele von Regression und Klassifikation generell?
</div>

<div class="alert block alert-success">
<b>Ihre Antwort:</b>
</div>

<div class="alert alert-block alert-success">
<b>Frage 4.4.17:</b> Was meinen Sie: Warum ist der Cross-Entropie-Verlust besser für Trainingsalgorithmen zur Klassifizierung geeignet?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


### One-Hot-Kodierung
Um eine Klassifizierung durchzuführen, müssen Kategorien in einer Weise dargestellt werden, die der Klassifikator verarbeiten kann. Neuronale Netze können Kategorien nicht direkt verstehen und benötigen eine numerische Darstellung.

#### Nachteile der Integer-Kodierung

Im Lama-Klassifikator wurde Lamas der Wert $1$ und Schafen der Wert $0$ zugewiesen. Ein einzelnes Ausgangsneuron würde "feuern", wenn ein Lama gefunden wurde, und nicht feuern, wenn ein Schaf gefunden wurde. Diese Art der Darstellung von Kategorien wird **ganzzahlige** (engl. integer) oder **Label-Kodierung** genannt.

Für die binäre Klassifikation funktioniert das recht gut, aber was ist, wenn wir zwischen Schafen, Lamas und Schäferhunden unterscheiden wollen?
Dies mit nur einem Ausgangsneuron zu tun, würde zu Komplikationen führen:
- Hunde bräuchten ein Label, das numerisch höher oder niedriger ist (z.B. $2$), was eine Ordnung impliziert (Hunde > Lamas), wo es eigentlich keine gibt.
- man müsste aus einem Wert des Ausgangsneurons drei verschiedene Zustände interpretieren

Ein weiterer Nachteil ist in der nächsten Frage zu sehen:

<div class="alert alert-block alert-success">
<b>Frage 4.4.18:</b> Angenommen, die Kodierungen sind: 0 für Schafe, 1 für Lamas und 2 für Hunde. Sie haben heute 5 Schafe und 5 Hunde klassifiziert. Sie möchten, dass Ihr Klassifikator die durchschnittliche Klassifikation für heute ausgibt. Was wird der Klassifikator ausgeben?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


#### Zusammensetzung der One-Hot-Kodierung

Die Lösung für die Mängel der Integer-Kodierung sieht wie folgt aus:

| Eingabe | One-Hot-Encoding |
|---------------|--------------------|
| Schaf | [1,0,0] |
| Lama | [0,1,0] |
| Hund | [0,0,1] |



Die Länge des Repräsentationsvektors ist immer gleich der Anzahl der Kategorien. Für jede Kategorie ist nur ein Element des Vektors 1 ("one-hot").
Mit dieser Kodierung können wir bequem 3 Ausgangsneuronen für 3 verschiedene Kategorien verwenden, so dass die Aktivierung jedes Ausgangsneurons den Klassifikationswert für diese Kategorie repräsentiert.

#### Grenzen der One-Hot-Kodierung
Die One-Hot-Kodierung ist keine unverbesserliche Lösung zur Darstellung von Kategorien, sondern eher ein weiteres Werkzeug in unserer Toolbox, das zufällig für viele Probleme gut funktioniert, aber nicht für alle.

<div class="alert alert-block alert-success">
<b>Frage 4.4.19:</b> Angenommen, Sie möchten ein neuronales Netzwerk zur Spracherkennung trainieren, das alle im Oxford English Dictionary enthaltenen englischen Wörter klassifizieren kann. Es muss nicht ganze Sätze klassifizieren, sondern nur einzelne Wörter. Was wäre ein Problem bei der Verwendung von One-Hot-Kodierung?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


### Softmax-Aktivierungsfunktion

Die Sigmoid-Funktion funktioniert gut für ein "Ja oder Nein"-Problem, d. h. für binäre Entscheidungen. Meistens wollen wir aber zwischen mehr als zwei Kategorien unterscheiden. Dafür brauchen wir eine Funktion, die **mehrere** Neuronenaktivierungen aus der letzten Schicht eines Netzes aufnimmt und einen **Wahrscheinlichkeitsvektor** ausgibt, der die Wahrscheinlichkeiten für jede Kategorie enthält.

Der Trick: **Jeder Eingang** dieser Funktion wird durch die anderen Eingänge **normalisiert**, so dass die Summe des Ausgangsvektors immer 1 ist. Diese Aktivierungsfunktion unterscheidet sich von ReLU oder Sigmoid, da sie immer für die gesamte Schicht gilt. In der Praxis ist sie nur als Aktivierungsfunktion für die Ausgabeschicht sinnvoll.  Abbildung 3 zeigt ein Beispielnetz.

Wir können eine Softmax-Aktivierungsfunktion realisieren, indem wir jedes Element $x_i$ des Eingangsvektors nehmen, $exp(x_i)$ berechnen und diesen Wert dann normalisieren, indem wir ihn durch die Summe der $exp$-Ergebnisse aller einzelnen Elemente des Eingangsvektors dividieren. Streng genommen ist das $exp$ für diesen Effekt nicht notwendig, eine lineare Normalisierung, beschränkt auf nicht-negative Werte, könnte auch als Wahrscheinlichkeit interpretiert werden. Allerdings bietet die exponentielle Normalisierung Eigenschaften, die die Performanz verbessern (siehe "Weiterführende Literatur").

\begin{align}
\phi_{\text{Softmax}}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}
\end{align}


<img src="https://github.com/KI-Campus/AMALEA/blob/master/Woche%204/images/softmax_example_network.png?raw=1" />
<p style="text-align: center;">
    Abb. 3 - Softmax-Aktivierungsfunktion
</p>

<div class="alert alert-block alert-success">
<b>Frage 4.4.20:</b> In der "logistischen Regression" haben wir auch eine Wahrscheinlichkeit erhalten, indem wir eine Sigmoidfunktion auf die Ausgabe der letzten Schichten angewendet haben. Warum können wir nicht eine Sigmoid-Funktion auf jedes Ausgangsneuron dieses Netzes anstelle eines Softmax anwenden und einen Wahrscheinlichkeitsvektor erhalten?
</div>

<div class="alert block alert-success">
<b>Ihre Antwort:</b>
</div>

***
## Automatisiertes Klassifizierungstraining

### Einleitung

Wir haben uns bereits im letzten Kapitel mit dem automatisierten Training mittels Backpropagation beschäftigt. Wir hatten einen Satz von Punkten, an die wir eine Funktion so gut wie möglich anpassen mussten. Beim Klassifikationstraining ist die Aufgabe ähnlich. Doch statt y-Koordinaten für Punkte haben wir nun diskrete Kategorien.

Sie haben bereits einen Satz von Halslängen und die dazugehörigen Kategorien (siehe Tabelle 1). Im Bereich des maschinellen Lernens wird dieser Datensatz __Trainingsdaten__ genannt. Er spezifiziert das Verhalten, das das neuronale Netz haben soll. Wir werden Backpropagation verwenden, um die Gewichte und Verzerrungen des Netzes immer wieder anzupassen, bis das Netz für einen gegebenen Satz von Eingaben die gleichen Werte ausgibt wie in den Trainingsdaten. Während der Backpropagation "lernt" das Netz im übertragenen Sinne die Trainingsdaten.

***
### Realisieren eines XOR-Gatter mit einem neuronalen Netz

Sie arbeiten als Ingenieur bei einer großen Firma, die elektronische Komponenten herstellt. Ihre Firma möchte den ersten XOR-Gatter-Chip herstellen, der mit künstlicher Intelligenz arbeitet. Sie erhalten die Trainingsdaten in Form einer Wahrheitstabelle:


| Eingang 1 | Eingang 2 | Ausgang |
|--------|----------|-----------|
| 0 | 0 |0 |
| 0 | 1 |1 |
| 1 | 0 |1 |
| 1 | 1 |0 |


<p style="text-align: center;">
    Tabelle. 2 - XOR-Wahrheitstabelle
</p>


In dieser Aufgabe werden wir Arrays und Matrizen verwenden, um den Umgang mit den Daten und den Netzwerkparametern zu erleichtern. Wir werden auch ein neuronales Netzwerk ohne Biases verwenden, um den Algorithmus so einfach wie möglich zu gestalten.
Die Trainingsdaten bestehen aus einem 2D-Array mit allen möglichen Eingangszuständen und einem 1D-Array mit allen entsprechenden Ausgängen.

#### Aufgabe : Trainingsdaten erstellen

Ein Trainingssatz besteht aus einem Eingabesatz und einem Lösungssatz. Beim überwachten Training wird das Netzwerk so lange angepasst, bis seine Vorhersagen für den Eingabesatz mit den entsprechenden vorgegebenen Lösungen übereinstimmen.
Vervollständigen Sie die Trainingsdaten unten mithilfe der Wahrheitstabelle

<div class="alert alert-block alert-success">
<b>Aufgabe 4.4.21:</b> Erstellen Sie die Trainingsdaten. Ein Trainingssatz besteht aus einem Eingabesatz und einem Lösungssatz. Beim überwachten Training wird das Netz so lange angepasst, bis seine Vorhersagen für den Eingabesatz mit den entsprechenden vorgegebenen Lösungen übereinstimmen (nicht immer, siehe: Overfitting, aber in diesem Fall). Vervollständigen Sie die untenstehenden Trainingsdaten unter Verwendung der obigen Groundtruth-Tabelle. Bitte initialisieren Sie auch den Lösungs-Array '2 dimensional'.

</div>

In [44]:
#xor_input_set = np.array(tablegoeshere)
#xor_solution_set = np.array(tablegoeshere)

# STUDENT CODE HERE
inputs = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

# Ausgabedaten: 4 Zeilen, 1 Spalte (Ausgang ist 1-dimensional, aber hier 2D-Array für Konsistenz)
outputs = np.array([
    [0],
    [1],
    [1],
    [0]
])
# STUDENT CODE until HERE


#### Initialisieren des Netzwerks
Als nächstes muss das Netzwerk definiert und initialisiert werden. Für diese Aufgabe verwenden wir ein Netz mit 3 versteckten Neuronen (siehe Abbildung 4).

<img src="https://github.com/KI-Campus/AMALEA/blob/master/Woche%204/images/3x2_xor_network.png?raw=1" />
<p style="text-align: center;">
    Abb. 4 - Neuronales Netzwerk
</p>

Wir definieren $w_{01}, w_{02}, w_{03}, w_{10}, w_{11}, w_{12}$ auf einmal, indem wir einfach eine 2x3-Gewichtsmatrix $w_{l1}$ definieren und das gleiche für $w_{l2}$ tun. Die Matrizen werden mit Werten zwischen -1 und 1 initialisiert

Führen Sie die untenstehende Zelle aus, um die neuronale Netzwerkklasse zu definieren, die oben abgebildet ist.

In [45]:
# do not change
class NeuralNetwork:
    def __init__(self):
        self.hl_sum = [0, 0, 0]
        self.hl_activation = [0, 0, 0]
        self.ol_sum = [0]
        self.prediction = 0
        self.b = 0
        self.w_i = np.zeros((2, 3))
        self.w_o = np.zeros((3, 1))

    def set_conf(self, w_i:float, w_o:float, b:float):  # w_i and w_o are matrices here
        self.w_i = w_i
        self.w_o = w_o
        self.b = b

    def get_conf(self)->dict:
        configuration = dict();
        configuration['w_i'] = self.w_i
        configuration['w_o'] = self.w_o
        configuration['b'] = self.b
        return configuration

    def get_ex(self)->dict:
        excitations = dict();
        excitations['hl_sum'] = self.hl_sum
        excitations['hl_activation'] = self.hl_activation
        excitations['ol_sum'] = self.ol_sum
        return excitations


    def show_conf(self):
        print("weight matrix w_i:")
        print(self.w_i)
        print("\nweight matrix w_o:")
        print(self.w_o)
        print("Bias")
        print(self.b)

    def compute(self, input_set:np.array)->float:
        self.hl_sum = input_set.dot(self.w_i)
        self.hl_activation = relu(self.hl_sum)
        self.ol_sum = relu(self.hl_activation).dot(self.w_o) + self.b
        self.prediction = sigmoid(self.ol_sum)

        return self.prediction

In [46]:
# do not change
logic_gate_net = NeuralNetwork()

In [47]:
# do not change
def initialize_network(net:NeuralNetwork):
    #np.random.seed(3)
    weight_matrix_i = np.random.rand(2,3)  # a 2x3 matrix of weights
    weight_matrix_o = np.random.rand(3,1)  # a 3x1 matrix of weights
    bias = np.random.randn()
    net.set_conf(weight_matrix_i,weight_matrix_o,bias)

In [48]:
# do not change
initialize_network(logic_gate_net) #just a test initialization to illustrate the weight matrices
logic_gate_net.show_conf()

weight matrix w_i:
[[0.14440104 0.3961311  0.00245241]
 [0.11186744 0.5708353  0.16318182]]

weight matrix w_o:
[[0.85031636]
 [0.19038445]
 [0.3777956 ]]
Bias
-0.04829199397204289


#### Definieren des Trainingsprozesses
Führen Sie abschließend die folgenden Zellen aus, um einen Backpropagation-Algorithmus zu implementieren. Versuchen Sie, den Code zu verstehen. Siehe Abb. 4 zur Erklärung der Variablennamen.

In [49]:
# do not change
def sigmoid_prime(x:float)->float: #the derivative of sigmoid
    return sigmoid(x)*(1-sigmoid(x))

In [50]:
# do not change
def train(net:NeuralNetwork, input_set:np.array, solution_set:np.array, learning_rate:float, epochs:int):
    for t in range(epochs):
        # Forward pass: compute predicted solution_set
        predictions = net.compute(input_set)
        # Compute and print loss
        log_loss = cross_entropy_loss(predictions, solution_set)

        if (t % 5 == 0):  # only output every 5th epoch
            print("Loss after Epoch {}: {:0.4f}".format(t, log_loss))

        #unravel variables here for readability
        ol_sum = net.get_ex()['ol_sum']
        hl_activation = net.get_ex()['hl_activation']
        hl_sum = net.get_ex()['hl_sum']
        w_i = net.get_conf()['w_i']
        w_o = net.get_conf()['w_o']
        b = net.get_conf()['b']

        # Backpropagation to compute gradients of w_i and w_o with respect to loss
        # start from the loss at the end and then work towards the front
        grad_ol_sum = sigmoid_prime(ol_sum) * (predictions - xor_solution_set)
        grad_w_o = hl_activation.T.dot(grad_ol_sum)  # Gradient of Loss with respect to w_o
        grad_hl_activation = grad_ol_sum.dot(w_o.T)  # the second layer's error
        grad_hl_sum = hl_sum.copy()  # create a copy to work with
        grad_hl_sum[hl_sum < 0] = 0  # the derivate of ReLU
        grad_w_i = input_set.T.dot(grad_hl_sum * grad_hl_activation)  #

        updated_weight_matrix_i = w_i - learning_rate * grad_w_i
        updated_weight_matrix_o = w_o - learning_rate * grad_w_o
        updated_bias = b - learning_rate * grad_ol_sum.sum()
        net.set_conf(updated_weight_matrix_i, updated_weight_matrix_o,
                       updated_bias)  # Apply updated weights to network

<div class="alert alert-block alert-success">
<b>Aufgabe 4.4.22:</b> Hyperparameter auswählen und trainieren
<ul>
<li> Wählen Sie eine optimale Lernrate und Anzahl der Epochen, indem Sie verschiedene Werte ausprobieren und die Zelle unten ausführen.
<li> Wenn Ihre Trainingsdaten korrekt waren, sollte das Netz nach dem Training einsatzbereit sein.
Ein erfolgreiches Training sollte zu einem Verlust kleiner als 0,02 führen.
                                                     
<li><b>Tipp:</b> Drücken Sie Umschalt+Eingabe auf der Zelle unten und dann die Pfeiltaste "nach oben", um das Training einfach zu wiederholen.

</ul>
</div>

In [61]:
learning_rate = 10
epochs = 1000
# STUDENT CODE HERE
xor_input_set = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

# Ausgabedaten: 4 Zeilen, 1 Spalte (Ausgang ist 1-dimensional, aber hier 2D-Array für Konsistenz)
xor_solution_set = np.array([
    [0],
    [1],
    [1],
    [0]
])
# STUDENT CODE until HERE

initialize_network(logic_gate_net) #initialize again so you can just run this box and train a new network
train(logic_gate_net, xor_input_set, xor_solution_set,learning_rate,epochs)

Loss after Epoch 0: 0.7917
Loss after Epoch 5: 0.7624
Loss after Epoch 10: 0.8167
Loss after Epoch 15: 0.4812
Loss after Epoch 20: 0.4806
Loss after Epoch 25: 0.4802
Loss after Epoch 30: 0.4799
Loss after Epoch 35: 0.4797
Loss after Epoch 40: 0.4796
Loss after Epoch 45: 0.4794
Loss after Epoch 50: 0.4793
Loss after Epoch 55: 0.4792
Loss after Epoch 60: 0.4791
Loss after Epoch 65: 0.4790
Loss after Epoch 70: 0.4790
Loss after Epoch 75: 0.4789
Loss after Epoch 80: 0.4789
Loss after Epoch 85: 0.4788
Loss after Epoch 90: 0.4788
Loss after Epoch 95: 0.4787
Loss after Epoch 100: 0.4787
Loss after Epoch 105: 0.4787
Loss after Epoch 110: 0.4786
Loss after Epoch 115: 0.4786
Loss after Epoch 120: 0.4786
Loss after Epoch 125: 0.4785
Loss after Epoch 130: 0.4785
Loss after Epoch 135: 0.4785
Loss after Epoch 140: 0.4785
Loss after Epoch 145: 0.4784
Loss after Epoch 150: 0.4784
Loss after Epoch 155: 0.4784
Loss after Epoch 160: 0.4784
Loss after Epoch 165: 0.4784
Loss after Epoch 170: 0.4784
Loss af

<div class="alert alert-block alert-success">
<b>Frage 4.4.23:</b> Warum sind die Verluste bei jeder Ausführung der Zelle anders?
</div>

<div class="alert block alert-success">
<b>Ihre Antwort:</b>
</div>

<div class="alert alert-block alert-success">
<b>Frage 4.4.24:</b> Was ist eine gute Lernrate, die in den meisten Fällen einen Verlust < 0,02 in < 100 Epochen erreicht?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Aufgabe 4.4.25:</b> Klassifizierungstest. Führen Sie die Zelle unten aus, verändern Sie die Stellungen der Schieberegler und führen Sie eine Validierungsprüfung auf Ihr Logikgatter aus.

</div>

In [63]:
# do not change
def change(input1:float, input2:float):
    input_vector = np.array([input1 * 1, input2 * 1])     # converting bool to float
    prediction = logic_gate_net.compute(input_vector)
    print("\t input: {} \t \t output: {:0.9f}".format(input_vector, prediction[0]))

interact(
    change,
    input1=FloatSlider(min=0, max=1, step=1, layout=Layout(width="22%")),
    input2=FloatSlider(min=0, max=1, step=1, layout=Layout(width="22%")),
);

interactive(children=(FloatSlider(value=0.0, description='input1', layout=Layout(width='22%'), max=1.0, step=1…

<div class="alert alert-block alert-success">
<b>Aufgabe 4.4.26:</b> Kontinuierlicher Eingangstest: Verändern Sie die Schieberegler Stellungen und beobachten Sie die Änderungen, wenn die Eingabe nicht binär, sondern kontinuierlich variiert wird.

</div>

In [64]:
interact(change, input1=0.0, input2=0.0);

interactive(children=(FloatSlider(value=0.0, description='input1', max=1.0), FloatSlider(value=0.0, descriptio…

<div class="alert alert-block alert-success">
<b>Frage 4.4.27:</b> Was können Sie beim Verändern der Schieberegler beobachten? Wie würden Sie den allgemeinen Zusammenhang zwischen den beiden Eingängen und dem Ausgang beschreiben (in wenigen Worten)
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 4.4.28:</b> Ändern Sie die Schieberegler auf die Werte der Trainingsdaten, z.B.(1.00, 1.00). Stimmt die Ausgabe genau mit den Trainingsdaten überein? Warum ist das der Fall?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b>
</div>

<div class="alert alert-block alert-success">
<b>Frage 4.4.29:</b> Das neuronale Netz kann jetzt etwas mehr vorhersagen, als nur die Werte der Eingabemenge, die Sie ihm gegeben haben. Welche "besondere Fähigkeit" hat Ihr Netz automatisch erlangt? (<b>Hinweis:</b> Denken Sie an neuronale Netze im Allgemeinen, das XOR-Gatter ist nur ein Beispiel)
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 4.4.30:</b> Wie kann diese spezielle Fähigkeit bei der Anwendung neuronaler Netze auf selbstfahrende Fahrzeuge nützlich sein?
</div>

<div class="alert block alert-success">
<b>Ihre Antwort:</b>
</div>

<div class="alert alert-block alert-success">
<b>Frage 4.4.31:</b> Warum macht es diese Fähigkeit einfacher, ein neuronales Netz für selbstfahrende Fahrzeuge zu verwenden als die traditionelle regelbasierte Programmierung (ein pos. und neg. Aspekt)?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>



<div class="alert alert-block alert-success">
<b>Aufgabe 4.4.32:</b> Erstellen Sie ein OR-Gatter. Ändern Sie den obigen Code, um ein OR-Netzwerk zu trainieren und überprüfen Sie Ihre Ergebnisse mit einem Test.

</div>

| Eingang 1 | Eingang 2 | Ausgang |
|--------|----------|-----------|
| 0 | 0 |0 |
| 0 | 1 |1 |
| 1 | 0 |1 |
| 1 | 1 |1 |


<p style="text-align: center;">
    Tabelle. 3 - ODER-Wahrheitstabelle
</p>

### Training in Keras

Nun soll das o.g. Beispiel in Keras implementiert werden.
Hierzu erstmals die benötigten Imports:

In [65]:
# benötigte Importe - nicht ändern
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import optimizers, losses

Als nächsten müssen die Daten für das Training vorbereitet werden.
Hierbei sollen zwei Listen entstehen, eine für die Datenpunkte (`train_data`) und eine für die dazugehörigen Labels (`train_labels`).
Beide Listen sollen die Daten von den Schaafen als auch der Lamas beinhalten.

<div class="alert alert-block alert-success">
<b>Aufgabe 4.4.33:</b> Erstellen Sie eine Liste mit dem Namen `train_data` für die Trainingsdatenpunkte aka Nackenhöhe sowie eine Liste mit dem Namen `train_labels` für die zugehörigen Labels, der Nackenhöhe der Schaffe sowie Lamas.
    </div>

In [66]:
print('Schafe: {}'.format(points_sheep))
print('Lamas: {}'.format(points_llamas))
# STUDENT CODE HERE

# STUDENT CODE until HERE
print('Datenpunkte für Training: {}'.format(train_data))
print('Labels für Training: {}'.format(train_labels))

Schafe: {'x': [0.2, 0.23, 0.28, 0.32, 0.35], 'y': [0, 0, 0, 0, 0]}
Lamas: {'x': [0.55, 0.68, 0.74, 0.83, 0.95], 'y': [1, 1, 1, 1, 1]}


NameError: name 'train_data' is not defined

Zuletzt muss noch das Modell vorbereitet werden.

<div class="alert alert-block alert-success">
<b>Aufgabe 4.4.34:</b> Erstellen Sie ein Model, bestehend aus zwei Dense Layern, wobei das erste drei Neuronen und das zweite ein Neuron besitzt und speichern dieses in einer Variablen mit dem Namen `model`. Als Aktivierungsfunktion soll ReLu und für die Ausgabe die Sigmoid-Funktion eingesetzt werden. Als Verlustfunktion soll der BinaryCrossentropy-Loss zum Einsatz kommen. Zuletzt soll RMSprop mit einer Lernrate von 0.01 als Optimierer verwendet werden.


_Hinweis: Verwenden Sie bei der Verlustfunktion <code>from_logits=False</code>_.
    </div>

In [ ]:
model = Sequential()
# STUDENT CODE HERE

# STUDENT CODE until HERE
print(model.summary())

<div class="alert alert-block alert-success">
    <b>Aufgabe 4.4.35:</b> Rufen Sie nun die <code>compile</code> Methode des Modells auf. Geben Sie hierbei als Metrik <i>accuracy</i> an.
    </div>    

In [ ]:
# STUDENT CODE HERE

# STUDENT CODE until HERE

<div class="alert alert-block alert-success">
<b>Aufgabe 4.4.36:</b> Starten Sie nun das Trainings ihres Models durch Aufruf der Fit-Funktion. Verwenden Sie hierbei eine  Batch Size von 2 und trainieren Sie 200 Epochen lang.
    </div>

In [ ]:
# STUDENT CODE HERE

# STUDENT CODE until HERE

In [ ]:
results = model.predict(train_data)
results = [1 if i > 0.5 else 0 for i in results]
correct_predicted = sum([1 if results[i] == train_labels[i] else 0 for i in range(len(results))])
false_predicted = len(results) - correct_predicted

print('Korrekt: {}\nNicht korrekt: {}'.format(correct_predicted, false_predicted))

### Ausblick: Klassifizierungstests in der realen Welt

Eine klassische Anwendung von neuronalen Netzen ist die Klassifizierung von Bildern. Ein häufig verwendeter Datensatz ist CIFAR-10, der aus folgenden Elementen besteht:  
 1. Bilder von Flugzeugen, Autos, Vögeln, Katzen, Rehen, Hunden, Fröschen, Pferden, Schiffen und Lastwagen (10 Kategorien)
 2. Labels, die an jedes Bild angehängt sind und das Bild kategorisieren

<img src="https://github.com/KI-Campus/AMALEA/blob/master/Woche%204/images/cifar10_plot.png?raw=1" />
<p style="text-align: center;">
    Abb. 3 - CIFAR-10-Datensatz[4]
</p>


Die Labels (auch Annotationen genannt) dienen als "Lösung" für den Trainingssatz. Jedes Element (Flugzeug, Auto...) ist eine eigene Kategorie.
Während des Trainings werden die Gewichte und Bias im Netzwerk genau so lange angepasst, bis das Modell die richtigen mathematischen Operationen ausführt, um die gegebenen Trainingsdaten korrekt zu klassifizieren. Nach dem Training kann das Netzwerk erkennen, ob es sich bei dem Bild um eine Katze, ein Flugzeug usw. handelt. Das funktioniert sogar bei Bildern, die das Netz noch nie gesehen hat. Wie neuronale Netze Bildklassifizierung durchführen, erfahren Sie in der nächsten Unterrichtseinheit.

#### Quellen:
[1] Wikipedia, Statistical classification https://en.wikipedia.org/wiki/Statistical_classification, retrieved 01.05.2019

[2]  Brownlee, Jason 2018. Machine Learning Algorithms From Scratch. p. 70

[3]  Gibbs, M.N. (Nov 2000). "Variational Gaussian process classifiers". IEEE Transactions on Neural Networks. p. 1458–1464.

[4] Cifar-10, Cifar-100 Dataset Introduction
Corochann - https://corochann.com/cifar-10-cifar-100-dataset-introduction-1258.html, retrieved 02.02.2019


#### Weiterführende Literatur:

The Sigmoid Function in Logistic Regression: http://karlrosaen.com/ml/notebooks/logistic-regression-why-sigmoid/

Why Softmax uses exponential function: https://stackoverflow.com/questions/17187507/why-use-softmax-as-opposed-to-standard-normalization